# Tratamento de dados - Secretaria de segurança do estado do Pará

## Descrição do notebook

O objetivo desse notebook é realizar o tratamento dos dados fornecidos pela Secretaria de Segurança Pública e Defesa Social do Estado do Pará para que possa ser analisado juntamente com os dados do estado paranaense.

Nessa versão, o arquivo utilizado é um XLSX e para manipulação de dados é utilizado o pacote **pandas**. 

**Observação:** O código contido nesse notebook foi baseado na solução desenvolvida por Mariana Kniss (https://www.linkedin.com/in/mariana-kniss-471ba0196/), para validar o conteúdo de seu trabalho de conclusão de curso, intitulado "ANÁLISE DE DADOS CRIMINAIS SOBRE VIOLÊNCIA CONTRA A MULHER NOS ESTADOS PARÁ E PARANÁ".

## Sumário

1. [Importação dos módulos e pacotes necessários](#importacao-dos-modulos-e-pacotes-necessarios)
2. [Carregamento dos dados](#carregamento-dos-dados)
4. [Visão geral do conjunto de dados](#visao-geral-do-conjunto-de-dados)
5. [Filtros, tratamentos e padronização de valores](#filtros-tratamentos-e-padronizacao-de-valores)
6. [Ordenação das colunas](#ordernacao-das-colunas)
7. [Exportação dos dados formatados](#exportacao-dos-dados-formatados)

----


## Importação dos módulos e pacotes necessários <a id="importacao-dos-modulos-e-pacotes-necessarios"></a>

In [1]:
# Pandas (https://pandas.pydata.org/)
import pandas as pd

# Datetime (https://docs.python.org/3/library/datetime.html)
import datetime as dt

## Carregamento dos dados

Inicialmente, realizada com **spark** e utilizando um arquivo do tipo **CSV**, o carregamento foi alterado para adotar o **pandas**, utilizando um arquivo do tipo **Excel (XLSX)**.

In [2]:
# Carregamento dos dados utilizando o arquivo XLSX e pandas
dados = pd.read_csv('./dados/PA/raw_BD_VD_2010a2022_PA.csv', sep='@')

/var/folders/ln/_mw0hd8d6ls5z_dsg4468zrh0000gp/T/ipykernel_3698/3921423267.py:2: DtypeWarning: Columns (35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  dados = pd.read_csv('./dados/PA/raw_BD_VD_2010a2022_PA.csv', sep='@')


In [3]:
# Exibe a quantidade de registros encontrados
len(dados)

196250

In [4]:
# Exibe o tipo de variável
type(dados)

pandas.core.frame.DataFrame

**Observação:** o método read_csv() do pandas carregou corretamente o número de registros: 196250.

----

## Visão geral do conjunto de dados <a id="visao-geral-do-conjunto-de-dados"></a>

In [5]:
# Contagem do número de variáveis
v = len(dados.columns)
# Contagem do número de registros
n = len(dados)

print(f'O conjunto de dados possui {n} registros e {v} variáveis.')

O conjunto de dados possui 196250 registros e 68 variáveis.


In [6]:
# Visualização do dataframe
dados.head(10)

,data_registro,hora_registro,data_fato,dia_semana,hora_fato,fx_4_hor,fx_12_hr,data_inst_proc,data_concl_proc,sit_proc,...,aut_profissao,aut_sit_emprego,aut_est_civil,meio_locomocao,cor_veiculo,marca_veic_fuga,modelo_do_veic_fuga,qtd_autor,relatorio,ident_autoria
0,2015-09-14,1970-01-01 20:25:00.000,2015-09-12,SAB,1970-01-01 11:40:00.000,06 |-- 12,10 |-- 12,1900-09-09,1900-09-09,NAO INSTAURADO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-11-25,1970-01-01 16:44:00.000,2022-11-25,SEX,1970-01-01 13:00:00.000,12 |-- 18,12 |-- 14,1900-06-06,1900-06-06,CONCLUSO,...,PREJUDICADO,PREJUDICADO,PREJUDICADO,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2,2022-11-25,1970-01-01 09:53:00.000,2022-05-28,SAB,1970-01-01 16:40:00.000,12 |-- 18,16 |-- 18,1900-06-06,1900-06-06,CONCLUSO,...,PREJUDICADO,PREJUDICADO,PREJUDICADO,NaN,NaN,NaN,NaN,1.0,NaN,NaN
3,2022-11-25,1970-01-01 09:32:00.000,2022-11-25,SEX,1970-01-01 08:00:00.000,06 |-- 12,08 |-- 10,1900-06-06,1900-06-06,CONCLUSO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4,2022-11-25,1970-01-01 08:47:00.000,2022-11-24,QUI,1970-01-01 21:00:00.000,18 |-- 24,20 |-- 22,1900-06-06,1900-06-06,CONCLUSO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
5,2022-11-24,1970-01-01 11:00:00.000,2022-11-20,DOM,1970-01-01 20:45:00.000,18 |-- 24,20 |-- 22,1900-06-06,1900-06-06,CONCLUSO,...,PREJUDICADO,PREJUDICADO,PREJUDICADO,NaN,NaN,NaN,NaN,1.0,NaN,NaN
6,2022-02-03,1970-01-01 11:08:59.999,2022-02-02,QUA,1970-01-01 22:00:00.000,18 |-- 24,22 |-- 24,1900-09-09,1900-09-09,CONCLUSO,...,PREJUDICADO,PREJUDICADO,PREJUDICADO,NaN,NaN,NaN,NaN,1.0,NaN,NaN
7,2022-11-25,1970-01-01 19:46:00.000,2022-11-25,SEX,1970-01-01 10:00:00.000,06 |-- 12,10 |-- 12,1900-06-06,1900-06-06,CONCLUSO,...,AUTONOMO,NaN,CASADO(A),A PE,NaN,NaN,NaN,1.0,NaN,NaN
8,2022-11-24,1970-01-01 09:04:00.000,2022-11-23,QUA,1970-01-01 23:55:00.000,18 |-- 24,22 |-- 24,1900-06-06,1900-06-06,CONCLUSO,...,PREJUDICADO,PREJUDICADO,PREJUDICADO,NaN,NaN,NaN,NaN,1.0,NaN,NaN
9,2022-11-24,1970-01-01 11:17:00.000,2022-11-22,TER,1970-01-01 20:00:00.000,18 |-- 24,20 |-- 22,1900-06-06,1900-06-06,CONCLUSO,...,PREJUDICADO,PREJUDICADO,PREJUDICADO,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [7]:
# Descrição dos tipos dos dados
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196250 entries, 0 to 196249
Data columns (total 68 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   data_registro           196250 non-null  object 
 1   hora_registro           196249 non-null  object 
 2   data_fato               196250 non-null  object 
 3   dia_semana              196250 non-null  object 
 4   hora_fato               196240 non-null  object 
 5   fx_4_hor                196250 non-null  object 
 6   fx_12_hr                196250 non-null  object 
 7   data_inst_proc          196250 non-null  object 
 8   data_concl_proc         196250 non-null  object 
 9   sit_proc                162210 non-null  object 
 10  classe_motivo           166521 non-null  object 
 11  mes_registro            196250 non-null  object 
 12  mes_fato                196250 non-null  object 
 13  ano_registro            196250 non-null  int64  
 14  ano_fato            

## Filtros, tratamentos e padronização de valores <a id="filtros-tratamentos-e-padronizacao-de-valores"></a>

### Filtros

#### Sexo das vítimas

In [8]:
# Contagem dos valores distintos na coluna 'vit_sexo'
dados.vit_sexo.value_counts()

vit_sexo
F     171066
M      16446
P       8241
PJ       496
JL         1
Name: count, dtype: int64

Filtrando apenas registros nos quais as vítimas são do sexo feminino.

In [9]:
# Filtrando a base para que "vit_sexo" = "F"
dados_filtro_vit_F = dados[dados['vit_sexo']  == 'F']

# Exibindo a quantidade de registros restantes
n_feminino = len(dados_filtro_vit_F)

print(f'Foram retirados {n - n_feminino}, restando {n_feminino} registros válidos.')

Foram retirados 25184, restando 171066 registros válidos.


#### Sexo dos autores

In [10]:
# Contagem dos valores distintos na coluna 'aut_sexo'
dados_filtro_vit_F.aut_sexo.value_counts()

aut_sexo
P     112068
M      47190
F       3272
PJ         1
Name: count, dtype: int64

Selecionando apenas regitros onde o sexo dos autores é diferente de feminino.

In [11]:
# Filtrando a base para que "aut_sexo" diferente "F"
dados_filtro_aut_nF = dados_filtro_vit_F.loc[(dados_filtro_vit_F['aut_sexo'] != 'F') | (dados_filtro_vit_F['aut_sexo'].isnull())]

# Exibindo o dataset VCM
n_autor_nf = len(dados_filtro_aut_nF)

print(f'Foram retirados {n_feminino - n_autor_nf}, restando {n_autor_nf} registros válidos.')

Foram retirados 3272, restando 167794 registros válidos.


**Comentários:** A partir desse procedimento, os valores obtidos para a utilização do pandas são discrepantes se comparados ao uso do pyspark. 

Mesmo assim, ao considerar que a quantidade de registros com o valor 'F' para a coluna 'aut_sexo' é 3272, ao subtrair esse valor dos 171066 registros válidos até esse ponto, o novo total seria 167794. Esse valor foi obtido pelo procedimento realizado aqui.

#### Ano do fato

A base de dados contava com registros de 2010 a 2023, porém foram selecionados apenas os casos ocorridos entre 2018 e 2022.

In [12]:
# Filtrando a base para que ano registro >= 2018
dados_filtro_ano_registro = dados_filtro_aut_nF[(dados_filtro_aut_nF['ano_fato'] >= 2018) & (dados_filtro_aut_nF['ano_fato'] <= 2022)]

# Exibindo o dataset VCM
n_registros_apos_2018 = len(dados_filtro_ano_registro)

print(f'Foram retirados {n_autor_nf - n_registros_apos_2018}, restando {n_registros_apos_2018} registros válidos.')

Foram retirados 64864, restando 102930 registros válidos.


#### Especificação do fato

Para selecionar apenas os registros de violência contra mulher (VCM), foram mantidas apenas as linhas que contivesse uma das expressões abaixo na coluna 'especificacao_fato':

- "mulher"
- "penha"
- "feminicídio"
- "femicídio"
- "feminicidio"
- "femicidio"

In [13]:
# Selecionando a base de VCM com base na variavel 'especializacao_fato', utilizando como filtro as palavras chave
palavras_chave = ["mulher", "feminicídio", "penha", "femicidio", "feminicidio", "femicídio"]

raw_VCM = dados_filtro_ano_registro[(dados_filtro_ano_registro['especializacao_fato'].notnull()) & (dados_filtro_ano_registro['especializacao_fato'].str.lower().str.contains('|'.join(palavras_chave)))]

In [14]:
n_vcm = len(raw_VCM)
print(f'Ao final, foram mantidos {n_vcm} registros. Ou seja, foram removidos {n_registros_apos_2018 - n_vcm} registros que não se encaixaram nos critérios definidos.')

Ao final, foram mantidos 7856 registros. Ou seja, foram removidos 95074 registros que não se encaixaram nos critérios definidos.


In [15]:
# Contagem dos valores distintos na coluna 'especializacao_fato'
raw_VCM['especializacao_fato'].value_counts()

especializacao_fato
CRIMES CONTRA A MULHER                   5103
CRIMES CONTRA A INTEGRIDADE DA MULHER    2743
VIOLENCIA DOMESTICA/MARIA DA PENHA         10
Name: count, dtype: int64

### Tratamentos

#### Removação de atributos não relevantes para as análises

In [16]:
# Remoção de colunas desnecessárias
colunas_para_remover = ['data_inst_proc', # Data de instauração do processo investigativo
						'data_concl_proc', # Data de conclusão do processo investigativo
						'sit_proc', # Situação do processo investigativo
						'grupo_ocorrencia', # Agrupamento (categorização) das ocorrências
						'sub_grupo', # Sub-agrupamento (categorização) das ocorrências
						'reg_integracao', # Regional de integração
						'risp', # RISP
						'aisp', # AISP
						'rua_fato', # Logradouro no qual aconteceu o fato
						'empresa', # Empresa de transporte relacionada ao fato
						'linha', # Linha de transporte relacionada ao fato
						'tipo_transporte', # Tipo de transporte relacionada ao fato
						'complemento', # Complemento do endereço
						'atuacao', # Atuação - Todos os valores constam como 'VITIMA'
						'meio_locomocao', # Meio de locomoção do autor
						'cor_veiculo', # Cor do veículo do autor
						'marca_veic_fuga', # Marca do veículo do autor
						'modelo_do_veic_fuga', # Modelo do veículo do autor
						'relatorio', # Relatório
						'qtd_autor', # Quantidade de autores relacionados à ocorrência
						'ident_autoria', # Se a identidade do autor é conhecida
						'consolidado', # Coluna que contém o valor 'HOMICIDIO' para todos os registros
						'fato_real', # Coluna que contém o valor 'HOMICIDIO' para todos os registros
						'meio_emp_deac', # Meio empregado, de acordo com a classificação da DEAC
						'causa_presumivel', # Causa presumível
						'especializacao_fato', # Especialização do fato
						'identificacao_fato', # Identificação do fato
						'vit_estado_civil', # Estado civil da vítima
						'aut_sexo', # Sexo do autor
						'aut_sit_emprego', # Situação empregatícia do autor
						'aut_est_civil' # Estado civil do autor
						]

raw_VCM = raw_VCM.drop(columns=colunas_para_remover)

In [17]:
# Contagem do número de variáveis
v_raw_vcm = len(raw_VCM.columns)
# Contagem do número de registros
n_raw_vcm = len(raw_VCM)

print(f'O conjunto de dados possui {n_raw_vcm} registros e {v_raw_vcm} variáveis.')

O conjunto de dados possui 7856 registros e 37 variáveis.


In [18]:
# Informações gerais do dataframe
raw_VCM.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7856 entries, 50 to 196236
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   data_registro           7856 non-null   object 
 1   hora_registro           7856 non-null   object 
 2   data_fato               7856 non-null   object 
 3   dia_semana              7856 non-null   object 
 4   hora_fato               7856 non-null   object 
 5   fx_4_hor                7856 non-null   object 
 6   fx_12_hr                7856 non-null   object 
 7   classe_motivo           7853 non-null   object 
 8   mes_registro            7856 non-null   object 
 9   mes_fato                7856 non-null   object 
 10  ano_registro            7856 non-null   int64  
 11  ano_fato                7856 non-null   int64  
 12  registros               7856 non-null   object 
 13  especificacao_crime     4928 non-null   object 
 14  latitude                483 non-null    ob

In [19]:
# Visualização do dataframe de VCM
raw_VCM.head(10)

,data_registro,hora_registro,data_fato,dia_semana,hora_fato,fx_4_hor,fx_12_hr,classe_motivo,mes_registro,mes_fato,...,vit_grau_inst,vit_profissao,vit_situacao_emprego,aut_data_nasc,aut_idade,aut_fx_etaria,grau_de_relacionamento,aut_cor_pele,aut_grau_inst,aut_profissao
50,2020-02-10,1970-01-01 13:26:00.000,2020-02-09,DOM,1970-01-01 13:26:00.000,12 |-- 18,12 |-- 14,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,FEVEREIRO,FEVEREIRO,...,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,1900-09-09 00:00:00,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN
51,2020-01-10,1970-01-01 16:34:00.000,2020-01-06,SEG,1970-01-01 16:00:00.000,12 |-- 18,16 |-- 18,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,JANEIRO,JANEIRO,...,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,1900-09-09 00:00:00,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN
52,2020-07-02,1970-01-01 09:56:00.000,2020-07-02,QUI,1970-01-01 09:56:00.000,06 |-- 12,08 |-- 10,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,JULHO,JULHO,...,ENSINO FUNDAMENTAL INCOMPLETO,DOMESTICA,NAO INFORMADO,1900-09-09 00:00:00,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN
82,2022-04-07,1970-01-01 16:47:00.000,2022-04-07,QUI,1970-01-01 16:47:00.000,12 |-- 18,16 |-- 18,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,ABRIL,ABRIL,...,NaN,NaN,NaN,NaN,66666.0,PREJUDICADO,NaN,PREJUDICADO,NaN,NaN
90,2022-03-16,1970-01-01 18:27:00.000,2022-03-16,QUA,1970-01-01 18:27:00.000,18 |-- 24,18 |-- 20,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,MARCO,MARCO,...,PREJUDICADO,PREJUDICADO,PREJUDICADO,1900-06-06 00:00:00,66666.0,PREJUDICADO,EX-COMPANHEIRO(A),PREJUDICADO,PREJUDICADO,PREJUDICADO
129,2021-02-15,1970-01-01 09:36:00.000,2021-02-14,DOM,1970-01-01 22:00:00.000,18 |-- 24,22 |-- 24,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,FEVEREIRO,FEVEREIRO,...,NAO INFORMADO,NAO INFORMADO,NaN,1900-09-09 00:00:00,66666.0,PREJUDICADO,COMPANHEIRO(A),NAO INFORMADO,NAO INFORMADO,NAO INFORMADO
182,2019-06-18,1970-01-01 17:05:00.000,2019-06-16,DOM,1970-01-01 15:05:00.000,12 |-- 18,14 |-- 16,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,JUNHO,JUNHO,...,NaN,NaN,NaN,1900-09-09 00:00:00,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN
227,2021-09-24,1970-01-01 14:46:00.000,2021-09-24,SEX,1970-01-01 14:46:00.000,12 |-- 18,14 |-- 16,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,SETEMBRO,SETEMBRO,...,NaN,NaN,NaN,NaN,NaN,PREJUDICADO,NaN,NaN,NaN,NaN
282,2019-06-14,1970-01-01 09:39:00.000,2019-06-05,QUA,1970-01-01 11:00:00.000,06 |-- 12,10 |-- 12,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,JUNHO,JUNHO,...,NaN,NaN,NaN,1900-09-09 00:00:00,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN
322,2021-07-18,1970-01-01 13:57:00.000,2021-07-18,DOM,1970-01-01 13:57:00.000,12 |-- 18,12 |-- 14,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,JULHO,JULHO,...,NaN,NaN,NaN,NaN,NaN,PREJUDICADO,NaN,NaN,NaN,NaN


#### Alteração dos tipos de variáveis

In [20]:
print(raw_VCM['hora_registro'].str.slice(11, 19))

50        13:26:00
51        16:34:00
52        09:56:00
82        16:47:00
90        18:27:00
            ...   
196220    11:34:00
196227    10:30:00
196232    11:18:00
196234    09:44:00
196236    09:09:00
Name: hora_registro, Length: 7856, dtype: object


In [21]:
# Converte as colunas necessárias para o formato de data
dados_VCM = raw_VCM
#dados_VCM.loc[:, 'data_registro'] = pd.to_datetime(dados_VCM['data_registro'])
#dados_VCM.loc[:, 'hora_registro'] = pd.to_datetime(raw_VCM['hora_registro'].astype('str').str.slice(11, 19), format='%H:%M:%S').dt.time
#dados_VCM.loc[:, 'data_fato'] = pd.to_datetime(dados_VCM['data_fato'])
#dados_VCM.loc[:, 'hora_fato'] = pd.to_datetime(raw_VCM['hora_fato'].astype('str').str.slice(11, 19), format='%H:%M:%S').dt.time
#dados_VCM.loc[:, 'vit_dt_nasc'] = pd.to_datetime(dados_VCM['vit_dt_nasc'])
#dados_VCM.loc[:, 'aut_data_nasc'] = pd.to_datetime(dados_VCM['aut_data_nasc'])

# Converte as colunas necessárias para o formato de data
dados_VCM = raw_VCM
dados_VCM['data_registro'] = pd.to_datetime(dados_VCM['data_registro'])
dados_VCM['hora_registro'] = pd.to_datetime(raw_VCM['hora_registro'].astype('str').str.slice(11, 19), format='%H:%M:%S').dt.time
dados_VCM['data_fato'] = pd.to_datetime(dados_VCM['data_fato'])
dados_VCM['hora_fato'] = pd.to_datetime(raw_VCM['hora_fato'].astype('str').str.slice(11, 19), format='%H:%M:%S').dt.time
dados_VCM['vit_dt_nasc'] = pd.to_datetime(dados_VCM['vit_dt_nasc'])
dados_VCM['aut_data_nasc'] = pd.to_datetime(dados_VCM['aut_data_nasc'])

In [22]:
dados_VCM.head(10)

,data_registro,hora_registro,data_fato,dia_semana,hora_fato,fx_4_hor,fx_12_hr,classe_motivo,mes_registro,mes_fato,...,vit_grau_inst,vit_profissao,vit_situacao_emprego,aut_data_nasc,aut_idade,aut_fx_etaria,grau_de_relacionamento,aut_cor_pele,aut_grau_inst,aut_profissao
50,2020-02-10,13:26:00,2020-02-09,DOM,13:26:00,12 |-- 18,12 |-- 14,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,FEVEREIRO,FEVEREIRO,...,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,1900-09-09,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN
51,2020-01-10,16:34:00,2020-01-06,SEG,16:00:00,12 |-- 18,16 |-- 18,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,JANEIRO,JANEIRO,...,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,1900-09-09,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN
52,2020-07-02,09:56:00,2020-07-02,QUI,09:56:00,06 |-- 12,08 |-- 10,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,JULHO,JULHO,...,ENSINO FUNDAMENTAL INCOMPLETO,DOMESTICA,NAO INFORMADO,1900-09-09,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN
82,2022-04-07,16:47:00,2022-04-07,QUI,16:47:00,12 |-- 18,16 |-- 18,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,ABRIL,ABRIL,...,NaN,NaN,NaN,NaT,66666.0,PREJUDICADO,NaN,PREJUDICADO,NaN,NaN
90,2022-03-16,18:27:00,2022-03-16,QUA,18:27:00,18 |-- 24,18 |-- 20,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,MARCO,MARCO,...,PREJUDICADO,PREJUDICADO,PREJUDICADO,1900-06-06,66666.0,PREJUDICADO,EX-COMPANHEIRO(A),PREJUDICADO,PREJUDICADO,PREJUDICADO
129,2021-02-15,09:36:00,2021-02-14,DOM,22:00:00,18 |-- 24,22 |-- 24,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,FEVEREIRO,FEVEREIRO,...,NAO INFORMADO,NAO INFORMADO,NaN,1900-09-09,66666.0,PREJUDICADO,COMPANHEIRO(A),NAO INFORMADO,NAO INFORMADO,NAO INFORMADO
182,2019-06-18,17:05:00,2019-06-16,DOM,15:05:00,12 |-- 18,14 |-- 16,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,JUNHO,JUNHO,...,NaN,NaN,NaN,1900-09-09,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN
227,2021-09-24,14:46:00,2021-09-24,SEX,14:46:00,12 |-- 18,14 |-- 16,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,SETEMBRO,SETEMBRO,...,NaN,NaN,NaN,NaT,NaN,PREJUDICADO,NaN,NaN,NaN,NaN
282,2019-06-14,09:39:00,2019-06-05,QUA,11:00:00,06 |-- 12,10 |-- 12,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,JUNHO,JUNHO,...,NaN,NaN,NaN,1900-09-09,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN
322,2021-07-18,13:57:00,2021-07-18,DOM,13:57:00,12 |-- 18,12 |-- 14,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,JULHO,JULHO,...,NaN,NaN,NaN,NaT,NaN,PREJUDICADO,NaN,NaN,NaN,NaN


#### Adição de uma nova coluna referente à idade da vítima

In [23]:
# Estatísticas descritivas da variáviel 'vit_idade'
dados_VCM['vit_idade'].describe()

count     7842.000000
mean      2672.337031
std      14471.063542
min          0.000000
25%         24.000000
50%         31.000000
75%         40.000000
max      99999.000000
Name: vit_idade, dtype: float64

In [24]:
# Substituindo os valores anômalos da variável 'vit_idade' por nulo
dados_VCM.loc[dados_VCM['vit_idade'] > 105, 'vit_idade'] = None

In [25]:
# Estatísticas descritivas da variáviel 'vit_idade' após a remoção dos valores anômalos
dados_VCM['vit_idade'].describe()

count    7575.000000
mean       32.354983
std        11.541125
min         0.000000
25%        24.000000
50%        31.000000
75%        39.000000
max       105.000000
Name: vit_idade, dtype: float64

In [26]:
# Se a data for menor ou igual a '1900-09-09', substitua por null, caso contrário, mantenha a data
dados_VCM.loc[dados_VCM['vit_dt_nasc'] <= '1900-09-09', 'vit_dt_nasc'] = None

In [27]:
# Calcule a idade em anos com tratamento para valores nulos
dados_VCM['vit_idade_calculada'] = (dados_VCM['data_fato'] - dados_VCM['vit_dt_nasc']).dt.days // 365

In [28]:
print(dados_VCM['vit_idade_calculada'].describe())
print(dados_VCM['vit_idade'].describe())

count    7528.000000
mean       32.423220
std        11.512158
min        -1.000000
25%        24.000000
50%        31.000000
75%        39.000000
max       105.000000
Name: vit_idade_calculada, dtype: float64
count    7575.000000
mean       32.354983
std        11.541125
min         0.000000
25%        24.000000
50%        31.000000
75%        39.000000
max       105.000000
Name: vit_idade, dtype: float64


#### Idade do autor

In [29]:
# Se a data for menor ou igual a '1900-09-09', substitua por null, caso contrário, mantenha a data
dados_VCM.loc[dados_VCM['aut_data_nasc'] <= '1900-09-09', 'aut_data_nasc'] = None

In [30]:
# Calcule a idade em anos com tratamento para valores nulos
dados_VCM['aut_idade_calculada'] = (dados_VCM['data_fato'] - dados_VCM['aut_data_nasc']).dt.days // 365

In [31]:
# Criando uma nova coluna referente à faixa etária
#dados_VCM['vit_idade_calculada'] = dados_VCM['vit_idade_calculada'].fillna('Desconhecida')

# Definindo os intervalos e os rótulos das faixas etárias
intervals = [0, 11, 17, 24, 29, 34, 64, 105]
labels = ['CRIANCA (0 A 11 ANOS)', 'ADOLESCENTE (12 A 17 ANOS)', 'ADULTO I (18 A 24 ANOS)', 'ADULTO II (25 A 29 ANOS)', 'ADULTO III (30 A 34 ANOS)', 'ADULTO IV (35 A 64 ANOS)', 'IDOSO']
categories = labels + ['Desconhecida']

dados_VCM['vit_faixa_etaria_calculada'] = pd.cut(dados_VCM['vit_idade_calculada'], bins=intervals, labels=labels, right=False)

In [32]:
# Adiciona a categoria 'Desconhecida' à lista de categorias da faixa etária
dados_VCM['vit_faixa_etaria_calculada'] = pd.Categorical(dados_VCM['vit_faixa_etaria_calculada'], categories=categories)
# Preenche os registros com valores nulos como 'Desconhecida'
dados_VCM['vit_faixa_etaria_calculada'] = dados_VCM['vit_faixa_etaria_calculada'].fillna('Desconhecida')

In [33]:
# Exibe a contagem das faixas etárias
dados_VCM['vit_faixa_etaria_calculada'].value_counts()

vit_faixa_etaria_calculada
ADULTO IV (35 A 64 ANOS)      2903
ADULTO I (18 A 24 ANOS)       1618
ADULTO II (25 A 29 ANOS)      1427
ADULTO III (30 A 34 ANOS)     1283
Desconhecida                   330
ADOLESCENTE (12 A 17 ANOS)     143
IDOSO                          128
CRIANCA (0 A 11 ANOS)           24
Name: count, dtype: int64

In [34]:
dados_VCM['vit_faixa_etaria_calculada'].describe()

count                         7856
unique                           8
top       ADULTO IV (35 A 64 ANOS)
freq                          2903
Name: vit_faixa_etaria_calculada, dtype: object

In [35]:
dados_VCM['aut_faixa_etaria_calculada'] = pd.cut(dados_VCM['aut_idade_calculada'], bins=intervals, labels=labels, right=False)

# Adiciona a categoria 'Desconhecida' à lista de categorias da faixa etária
dados_VCM['aut_faixa_etaria_calculada'] = pd.Categorical(dados_VCM['aut_faixa_etaria_calculada'], categories=categories)
# Preenche os registros com valores nulos como 'Desconhecida'
dados_VCM['aut_faixa_etaria_calculada'] = dados_VCM['aut_faixa_etaria_calculada'].fillna('Desconhecida')

In [36]:
# Exibe a contagem das faixas etárias
dados_VCM['aut_faixa_etaria_calculada'].value_counts()

aut_faixa_etaria_calculada
Desconhecida                  7761
ADULTO IV (35 A 64 ANOS)        40
ADULTO II (25 A 29 ANOS)        21
ADULTO I (18 A 24 ANOS)         19
ADULTO III (30 A 34 ANOS)       14
ADOLESCENTE (12 A 17 ANOS)       1
CRIANCA (0 A 11 ANOS)            0
IDOSO                            0
Name: count, dtype: int64

In [37]:
# Verifica a quantidade de casos nos quais as vítimas tinham mais de 18 anos
dados_VCM_maiores_18_PA = dados_VCM[dados_VCM['vit_idade_calculada'] >= 18]
len(dados_VCM_maiores_18_PA)

7277

In [38]:
# Verifica a quantidade total de registros
len(dados_VCM)

7856

In [39]:
# Cria um novo dataframe para ser salvo referente aos dados do estado do Pará
dados_VCM_PA = dados_VCM
# Visualização do dataframe
dados_VCM_PA.head(10)

,data_registro,hora_registro,data_fato,dia_semana,hora_fato,fx_4_hor,fx_12_hr,classe_motivo,mes_registro,mes_fato,...,aut_idade,aut_fx_etaria,grau_de_relacionamento,aut_cor_pele,aut_grau_inst,aut_profissao,vit_idade_calculada,aut_idade_calculada,vit_faixa_etaria_calculada,aut_faixa_etaria_calculada
50,2020-02-10,13:26:00,2020-02-09,DOM,13:26:00,12 |-- 18,12 |-- 14,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,FEVEREIRO,FEVEREIRO,...,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN,60.0,NaN,ADULTO IV (35 A 64 ANOS),Desconhecida
51,2020-01-10,16:34:00,2020-01-06,SEG,16:00:00,12 |-- 18,16 |-- 18,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,JANEIRO,JANEIRO,...,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN,39.0,NaN,ADULTO IV (35 A 64 ANOS),Desconhecida
52,2020-07-02,09:56:00,2020-07-02,QUI,09:56:00,06 |-- 12,08 |-- 10,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,JULHO,JULHO,...,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN,40.0,NaN,ADULTO IV (35 A 64 ANOS),Desconhecida
82,2022-04-07,16:47:00,2022-04-07,QUI,16:47:00,12 |-- 18,16 |-- 18,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,ABRIL,ABRIL,...,66666.0,PREJUDICADO,NaN,PREJUDICADO,NaN,NaN,23.0,NaN,ADULTO I (18 A 24 ANOS),Desconhecida
90,2022-03-16,18:27:00,2022-03-16,QUA,18:27:00,18 |-- 24,18 |-- 20,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,MARCO,MARCO,...,66666.0,PREJUDICADO,EX-COMPANHEIRO(A),PREJUDICADO,PREJUDICADO,PREJUDICADO,33.0,NaN,ADULTO III (30 A 34 ANOS),Desconhecida
129,2021-02-15,09:36:00,2021-02-14,DOM,22:00:00,18 |-- 24,22 |-- 24,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,FEVEREIRO,FEVEREIRO,...,66666.0,PREJUDICADO,COMPANHEIRO(A),NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,27.0,NaN,ADULTO II (25 A 29 ANOS),Desconhecida
182,2019-06-18,17:05:00,2019-06-16,DOM,15:05:00,12 |-- 18,14 |-- 16,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,JUNHO,JUNHO,...,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN,13.0,NaN,ADOLESCENTE (12 A 17 ANOS),Desconhecida
227,2021-09-24,14:46:00,2021-09-24,SEX,14:46:00,12 |-- 18,14 |-- 16,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,SETEMBRO,SETEMBRO,...,NaN,PREJUDICADO,NaN,NaN,NaN,NaN,31.0,NaN,ADULTO III (30 A 34 ANOS),Desconhecida
282,2019-06-14,09:39:00,2019-06-05,QUA,11:00:00,06 |-- 12,10 |-- 12,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,JUNHO,JUNHO,...,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN,21.0,NaN,ADULTO I (18 A 24 ANOS),Desconhecida
322,2021-07-18,13:57:00,2021-07-18,DOM,13:57:00,12 |-- 18,12 |-- 14,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,JULHO,JULHO,...,NaN,PREJUDICADO,NaN,NaN,NaN,NaN,38.0,NaN,ADULTO IV (35 A 64 ANOS),Desconhecida


#### Tratamento dos meses

In [40]:
# Dicionário de mapeamento de meses para números
meses = {
    "JANEIRO": 1,
    "FEVEREIRO": 2,
    "MARCO": 3,
    "ABRIL": 4,
    "MAIO": 5,
    "JUNHO": 6,
    "JULHO": 7,
    "AGOSTO": 8,
    "SETEMBRO": 9,
    "OUTUBRO": 10,
    "NOVEMBRO": 11,
    "DEZEMBRO": 12
}

dados_VCM_PA['mes_registro'] = dados_VCM_PA['mes_registro'].replace(meses)
dados_VCM_PA['mes_fato'] = dados_VCM_PA['mes_fato'].replace(meses)

In [41]:
dados_VCM_PA.head(10)

,data_registro,hora_registro,data_fato,dia_semana,hora_fato,fx_4_hor,fx_12_hr,classe_motivo,mes_registro,mes_fato,...,aut_idade,aut_fx_etaria,grau_de_relacionamento,aut_cor_pele,aut_grau_inst,aut_profissao,vit_idade_calculada,aut_idade_calculada,vit_faixa_etaria_calculada,aut_faixa_etaria_calculada
50,2020-02-10,13:26:00,2020-02-09,DOM,13:26:00,12 |-- 18,12 |-- 14,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,2,2,...,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN,60.0,NaN,ADULTO IV (35 A 64 ANOS),Desconhecida
51,2020-01-10,16:34:00,2020-01-06,SEG,16:00:00,12 |-- 18,16 |-- 18,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,1,1,...,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN,39.0,NaN,ADULTO IV (35 A 64 ANOS),Desconhecida
52,2020-07-02,09:56:00,2020-07-02,QUI,09:56:00,06 |-- 12,08 |-- 10,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,7,7,...,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN,40.0,NaN,ADULTO IV (35 A 64 ANOS),Desconhecida
82,2022-04-07,16:47:00,2022-04-07,QUI,16:47:00,12 |-- 18,16 |-- 18,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,4,4,...,66666.0,PREJUDICADO,NaN,PREJUDICADO,NaN,NaN,23.0,NaN,ADULTO I (18 A 24 ANOS),Desconhecida
90,2022-03-16,18:27:00,2022-03-16,QUA,18:27:00,18 |-- 24,18 |-- 20,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,3,3,...,66666.0,PREJUDICADO,EX-COMPANHEIRO(A),PREJUDICADO,PREJUDICADO,PREJUDICADO,33.0,NaN,ADULTO III (30 A 34 ANOS),Desconhecida
129,2021-02-15,09:36:00,2021-02-14,DOM,22:00:00,18 |-- 24,22 |-- 24,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,2,2,...,66666.0,PREJUDICADO,COMPANHEIRO(A),NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,27.0,NaN,ADULTO II (25 A 29 ANOS),Desconhecida
182,2019-06-18,17:05:00,2019-06-16,DOM,15:05:00,12 |-- 18,14 |-- 16,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,6,6,...,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN,13.0,NaN,ADOLESCENTE (12 A 17 ANOS),Desconhecida
227,2021-09-24,14:46:00,2021-09-24,SEX,14:46:00,12 |-- 18,14 |-- 16,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,9,9,...,NaN,PREJUDICADO,NaN,NaN,NaN,NaN,31.0,NaN,ADULTO III (30 A 34 ANOS),Desconhecida
282,2019-06-14,09:39:00,2019-06-05,QUA,11:00:00,06 |-- 12,10 |-- 12,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,6,6,...,99999.0,NAO INFORMADO,NaN,NaN,NaN,NaN,21.0,NaN,ADULTO I (18 A 24 ANOS),Desconhecida
322,2021-07-18,13:57:00,2021-07-18,DOM,13:57:00,12 |-- 18,12 |-- 14,CRIMES DE VIOLENCIA DOMESTICA - LEI 11.340/2006,7,7,...,NaN,PREJUDICADO,NaN,NaN,NaN,NaN,38.0,NaN,ADULTO IV (35 A 64 ANOS),Desconhecida


#### Faixa horária

In [42]:
horas = dados_VCM_PA['hora_fato']

for h in horas:
	print(h.hour)
	break

13


In [43]:

# Definindo os intervalos e os rótulos das faixas etárias
intervals = [0, 6, 12, 18, 24]
labels = ['Madrugada', 'Manhã', 'Tarde', 'Noite']
categories = labels + ['Outro']

# Cria uma nova coluna com a hora do fato no formato inteiro
dados_VCM_PA['hora_fato_int'] = dados_VCM_PA['hora_fato'].apply(lambda x: x.hour)

# Período que ocorreu o fato (Manhã, Tarde, Noite ou Madrugada)
dados_VCM_PA['faixa_horaria'] = pd.cut(dados_VCM_PA['hora_fato_int'], bins=intervals, labels=labels, right=False)
dados_VCM_PA['faixa_horaria'] = pd.Categorical(dados_VCM['faixa_horaria'], categories=categories)
dados_VCM_PA['faixa_horaria'] = dados_VCM['faixa_horaria'].fillna('Outro')

# Período que ocorreu o fato (['00h - 06h', '06h - 12h', '12h - 18h' ou '18h - 00h')
labels = ['00h - 06h', '06h - 12h', '12h - 18h', '18h - 00h']
categories = labels + ['Outro']

dados_VCM_PA['faixa_horaria_6h'] = pd.cut(dados_VCM_PA['hora_fato_int'], bins=intervals, labels=labels, right=False)
dados_VCM_PA['faixa_horaria_6h'] = pd.Categorical(dados_VCM['faixa_horaria_6h'], categories=categories)
dados_VCM_PA['faixa_horaria_6h'] = dados_VCM['faixa_horaria_6h'].fillna('Outro')

intervals = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]
labels = ['00h - 02h', '02h - 04h', '04h - 06h', '06h - 08h', '08h - 10h', '10h - 12h', '12h - 14h', '14h - 16h', '16h - 18h', '18h - 20h', '20h - 22h', '22h - 00h']
categories = labels + ['Outro']

dados_VCM_PA['faixa_horaria_2h'] = pd.cut(dados_VCM_PA['hora_fato_int'], bins=intervals, labels=labels, right=False)
dados_VCM_PA['faixa_horaria_2h'] = pd.Categorical(dados_VCM['faixa_horaria_2h'], categories=categories)
dados_VCM_PA['faixa_horaria_2h'] = dados_VCM['faixa_horaria_2h'].fillna('Outro')

# Exibindo a coluna com o horário e as demais colunas criadas
dados_VCM_PA[['hora_fato', 'hora_fato_int', 'faixa_horaria', 'faixa_horaria_6h', 'faixa_horaria_2h']].head(10)

,hora_fato,hora_fato_int,faixa_horaria,faixa_horaria_6h,faixa_horaria_2h
50,13:26:00,13,Tarde,12h - 18h,12h - 14h
51,16:00:00,16,Tarde,12h - 18h,16h - 18h
52,09:56:00,9,Manhã,06h - 12h,08h - 10h
82,16:47:00,16,Tarde,12h - 18h,16h - 18h
90,18:27:00,18,Noite,18h - 00h,18h - 20h
129,22:00:00,22,Noite,18h - 00h,22h - 00h
182,15:05:00,15,Tarde,12h - 18h,14h - 16h
227,14:46:00,14,Tarde,12h - 18h,14h - 16h
282,11:00:00,11,Manhã,06h - 12h,10h - 12h
322,13:57:00,13,Tarde,12h - 18h,12h - 14h


In [44]:
# Remoção de mais colunas
colunas_para_remover = ['fx_4_hor',
					'fx_12_hr',
					'latitude',
					'longitude',
					'distrito',
					'regionais',
					'vit_idade',
					'vit_fx_etaria',
					'vit_sexo',
					'vit_situacao_emprego',
					'aut_idade',
					'aut_fx_etaria',
					'hora_fato_int'
				]

dados_VCM_PA = dados_VCM_PA.drop(columns=colunas_para_remover)

In [45]:
# Contagem do número de variáveis
v_vcm_pa = len(dados_VCM_PA.columns)
# Contagem do número de registros
n_vcm_pa = len(dados_VCM_PA)

print(f'O conjunto de dados possui {n_vcm_pa} registros e {v_vcm_pa} variáveis.')

O conjunto de dados possui 7856 registros e 32 variáveis.


## Ordenação das colunas <a id="ordernacao-das-colunas"></a>

### Renomeando colunas

In [46]:
dados_VCM_PA = dados_VCM_PA.rename(columns={
	'vit_idade_calculada'		: 'vit_idade',
	'aut_idade_calculada'		: 'aut_idade',
	'vit_faixa_etaria_calculada': 'vit_faixa_etaria',
	'aut_faixa_etaria_calculada': 'aut_faixa_etaria',
	'meio_empregado_sisp'		: 'meio_empregado',
	'municipios'				: 'municipio',
	'bairros'					: 'bairro',
	'aut_data_nasc'				: 'aut_dt_nasc'})

Cria uma nova coluna para especificar o estado do Pará.

In [47]:
dados_VCM_PA['estado'] = 'PA'

In [48]:
len(dados_VCM_PA.columns)

33

### Definindo a nova ordem das colunas

In [49]:
dados_VCM_PA = dados_VCM_PA[['data_registro', 'hora_registro', 'data_fato', 'hora_fato', 'dia_semana', 'faixa_horaria_2h', 'faixa_horaria_6h', 'classe_motivo', 'mes_fato', 'ano_registro', 'ano_fato', 'registros', 'municipio', 'bairro', 'local_ocorrencia', 'meio_empregado', 'mes_registro', 'especificacao_crime', 'vit_dt_nasc', 'vit_idade', 'vit_faixa_etaria', 'vit_cor_pele', 'vit_grau_inst', 'vit_profissao', 'aut_dt_nasc', 'aut_idade', 'aut_faixa_etaria', 'aut_cor_pele', 'aut_grau_inst', 'aut_profissao', 'grau_de_relacionamento', 'faixa_horaria', 'estado']]

## Exportação dos dados formatados <a id="exportacao-dos-dados-formatados"></a>

In [50]:
import datetime as dt # Manipulação de data

dados_VCM_PA.to_csv('./dados-tratados/' + dt.datetime.now().strftime('%Y%m%d%H%M%S') + '-dados_VCM_PA-pandas.csv', index=False, sep='|', encoding='utf-8')